<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Développement-de-notre-propre-technique-d'évaluation-d'une-réponse" data-toc-modified-id="Développement-de-notre-propre-technique-d'évaluation-d'une-réponse-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Développement de notre propre technique d'évaluation d'une réponse</a></span><ul class="toc-item"><li><span><a href="#Quelques-exemples-de-réponses..." data-toc-modified-id="Quelques-exemples-de-réponses...-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Quelques exemples de réponses...</a></span></li><li><span><a href="#Cleaning" data-toc-modified-id="Cleaning-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Cleaning</a></span></li><li><span><a href="#Vectorisation" data-toc-modified-id="Vectorisation-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Vectorisation</a></span></li><li><span><a href="#Cosine-similarity" data-toc-modified-id="Cosine-similarity-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Cosine similarity</a></span></li><li><span><a href="#Distance-2-à-2" data-toc-modified-id="Distance-2-à-2-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>Distance 2 à 2</a></span></li><li><span><a href="#Wrapping-up" data-toc-modified-id="Wrapping-up-1.6"><span class="toc-item-num">1.6&nbsp;&nbsp;</span>Wrapping up</a></span></li></ul></li><li><span><a href="#Utilisation-de-package-haystack-pour-évaluer-les-questions/réponses-d'un-modèle" data-toc-modified-id="Utilisation-de-package-haystack-pour-évaluer-les-questions/réponses-d'un-modèle-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Utilisation de package haystack pour évaluer les questions/réponses d'un modèle</a></span><ul class="toc-item"><li><span><a href="#Importer-les-données-de-test" data-toc-modified-id="Importer-les-données-de-test-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Importer les données de test</a></span></li><li><span><a href="#Importer-un-modèle-et-évaluer-sur-toutes-les-questions/réponses-de-test." data-toc-modified-id="Importer-un-modèle-et-évaluer-sur-toutes-les-questions/réponses-de-test.-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Importer un modèle et évaluer sur toutes les questions/réponses de test.</a></span></li></ul></li></ul></div>

# Développement de notre propre technique d'évaluation d'une réponse

In [ ]:
import nltk
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
import string
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords

In [ ]:
stopwords = stopwords.words('french')

## Quelques exemples de réponses...

La premier élément de la liste est la réponse "vraie" i.e. celle attendue et renseigné dans les fichiers `json`. Les autres éléments de la liste servent d'exemple de réponse qui pourraient être renvoyées par le Chatbot.

In [ ]:
response = [
            "remplacer l’hydrogène fossile (issu à 95% du gaz, pétrole et charbon) actuellement utilisé par l’industrie dans ses process en France (dans le secteur du raffinage, de la production d’ammoniac ou de la chimie) par de l’hydrogène bas-carbone",
            "remplacer les hydrocarbures",
            "hydrogène bas-carbone",
            "ne pas utiliser d'hydrogène bas-carbone"
]

## Cleaning
Retrait de la ponctuation et des mots de fin.

In [ ]:
def clean_string(text: str):
    text = ''.join([w for w in text if w not in string.punctuation])
    text = text.lower()
    text = ' '.join([w for w in text.split() if w not in stopwords])

    return text

In [ ]:
response_clean = list(map(clean_string, response))
response_clean

['remplacer l’hydrogène fossile issu 95 gaz pétrole charbon actuellement utilisé l’industrie process france secteur raffinage production d’ammoniac chimie l’hydrogène bascarbone',
 'remplacer hydrocarbures',
 'hydrogène bascarbone',
 'utiliser dhydrogène bascarbone']

## Vectorisation
Vectorisation et création de la matrice des tokens. 

In [ ]:
vectorizer = CountVectorizer()
vect = vectorizer.fit_transform(response_clean).toarray()
vectorizer.get_feature_names_out()

array(['95', 'actuellement', 'ammoniac', 'bascarbone', 'charbon',
       'chimie', 'dhydrogène', 'fossile', 'france', 'gaz',
       'hydrocarbures', 'hydrogène', 'industrie', 'issu', 'process',
       'production', 'pétrole', 'raffinage', 'remplacer', 'secteur',
       'utiliser', 'utilisé'], dtype=object)

In [ ]:
vect[0]

array([1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 2, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1])

## Cosine similarity
Calcul de la matrice des distances

In [ ]:
cosine_similarity(vect)

array([[1.        , 0.15075567, 0.45226702, 0.12309149],
       [0.15075567, 1.        , 0.        , 0.        ],
       [0.45226702, 0.        , 1.        , 0.40824829],
       [0.12309149, 0.        , 0.40824829, 1.        ]])

## Distance 2 à 2
Reprises des calculs de distances: renvoyer une distance pour uniquement deux réponses

In [ ]:
def simi_cosim(va, vb):
    va = va.reshape(1, -1)
    vb = vb.reshape(1, -1)

    return cosine_similarity(va, vb)[0][0]

In [ ]:
simi_cosim(vect[0], vect[2])

0.4522670168666454

## Wrapping up  

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
import string
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords

stopwords = stopwords.words('french')

def eval_similirity(out_resp: str, exp_resp: str):
    """ Computes cosine similarity between 2 answers

    Args:
      out_resp (str): the response to be evaluated
      exp_resp (str): the response expexted

    Returns:
      similarity (float): the cosine similirity computed between out_resp and
      exp_resp
    """

    out_resp = ''.join([w for w in out_resp if w not in string.punctuation])
    exp_resp = ''.join([w for w in exp_resp if w not in string.punctuation])

    out_resp = out_resp.lower()
    exp_resp = exp_resp.lower()

    out_resp = ' '.join([w for w in out_resp.split() if w not in stopwords])
    exp_resp = ' '.join([w for w in exp_resp.split() if w not in stopwords])

    
    vectorizer = CountVectorizer()
    vect = vectorizer.fit_transform([exp_resp, out_resp]).toarray()

    return cosine_similarity(vect[0].reshape(1, -1), vect[1].reshape(1, -1))[0][0]

In [ ]:
eval_similirity("this is the first document we found", "third document")

0.26726124191242434

# Utilisation de package haystack pour évaluer les questions/réponses d'un modèle 

## Importer les données de test

In [2]:
import json 

# Opening JSON file
f = open("/content/drive/MyDrive/Projet Option - ChatBot/3 - Scripts/Question-Réponse/data_test_squad_format.json")
 
# returns JSON object as
# a dictionary
data_test = json.load(f)


In [ ]:
data_test["data"][0]

{'paragraphs': [{'context': 'Si les défis technologiques et de R&D associés apparaissent «dépassables» dans les décennies à venir, les scénarios «100% renouvelable» ou fondés sur la prolongation à long terme des réacteurs nucléaires actuels au-delà de 60 ans impliquent qu’un grand nombre de prérequis techniques critiques soient respectés à court terme. Or rien ne le garantit en l’état. Décider de ces scénarios aujourd’hui, ou renoncer au principe de diversification technologique dans le mix de production électrique, soulève donc un risque de non-atteinte de l’objectif de neutralité carbone à la date rapprochée de 2050.',
   'document_id': 15,
   'qas': [{'answers': [{'answer_category': 'SHORT',
       'answer_id': 5015,
       'answer_start': 247,
       'document_id': 15,
       'question_id': 1015,
       'text': 'grand nombre de prérequis techniques critiques soient respectés à court terme'}],
     'id': 1015,
     'is_impossible': False,
     'question': 'Que sous-entendent les scé

In [ ]:
len(data_test["data"])

60

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Importer un modèle et évaluer sur toutes les questions/réponses de test. 

In [ ]:
!pip install git+https://github.com/deepset-ai/haystack.git#egg=farm-haystack[colab]

In [11]:
import json
from haystack.utils import convert_files_to_dicts, fetch_archive_from_http, print_answers
from haystack.nodes import FARMReader, TransformersReader
from haystack.document_stores import ElasticsearchDocumentStore
from haystack.nodes import ElasticsearchRetriever, FARMReader
from haystack.pipelines import ExtractiveQAPipeline
from pathlib import Path
import pickle
from time import sleep


# #############################################
# ## Launching and setting up pipe obj       ##
# #############################################
from haystack.utils import launch_es

! wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.9.2-linux-x86_64.tar.gz -q
! tar -xzf elasticsearch-7.9.2-linux-x86_64.tar.gz
! chown -R daemon:daemon elasticsearch-7.9.2

import os
from subprocess import Popen, PIPE, STDOUT

es_server = Popen(
    ["elasticsearch-7.9.2/bin/elasticsearch"], stdout=PIPE, stderr=STDOUT, preexec_fn=lambda: os.setuid(1)  # as daemon
)
# wait until ES has started
! sleep 30


# Connect to Elasticsearch
document_store = ElasticsearchDocumentStore(host="localhost", username="", password="", index="document")
report_path = "/content/drive/MyDrive/Projet Option - ChatBot/0 - Biblio/rapport_rte_v2.json"

# Import text 
with open(report_path) as json_file:
    dict_rapport = json.load(json_file)["data"]

# write document on ElasticSearch server
document_store.write_documents(dict_rapport)

## Initalize Retriever, Reader,  & Pipeline
retriever = ElasticsearchRetriever(document_store=document_store)
reader = FARMReader(model_name_or_path="/content/drive/MyDrive/Projet Option - ChatBot/3 - Scripts/camembertV2", use_gpu=True)
# reader = FARMReader(model_name_or_path="saattrupdan/xlmr-base-texas-squad-fr", use_gpu=True)

pipe = ExtractiveQAPipeline(reader, retriever)


INFO - haystack.modeling.utils -  Using devices: CUDA
INFO - haystack.modeling.utils -  Number of GPUs: 1
INFO - haystack.modeling.model.language_model -  LOADING MODEL
INFO - haystack.modeling.model.language_model -  =============
INFO - haystack.modeling.model.language_model -  Model found locally at /content/drive/MyDrive/Projet Option - ChatBot/3 - Scripts/camembertV2
INFO - haystack.modeling.model.language_model -  Loaded /content/drive/MyDrive/Projet Option - ChatBot/3 - Scripts/camembertV2
INFO - haystack.modeling.model.adaptive_model -  Found files for loading 1 prediction heads
WARNING - haystack.modeling.model.prediction_head -  Some unused parameters are passed to the QuestionAnsweringHead. Might not be a problem. Params: {"training": true, "num_labels": 2, "ph_output_type": "per_token_squad", "model_type": "span_classification", "label_tensor_name": "question_answering_label_ids", "label_list": ["start_token", "end_token"], "metric": "squad", "name": "QuestionAnsweringHead"

Evaluer sur chaque question/réponse (on évalue sur la capacité à trouver la réponse la plus proche sur les 3 réponses):

In [5]:
import time 
time.time()-time.time()

-2.384185791015625e-07

In [12]:

# We can load evaluation labels from the document store
# We are also opting to filter out no_answer samples
# eval_labels = document_store.get_all_labels_aggregated(drop_negative_labels=True, drop_no_answers=False)
# eval_labels = [label for label in eval_labels if not label.no_answer]  # filter out no_answer cases

## Alternative: Define queries and labels directly
from haystack.schema import EvaluationResult, MultiLabel, Label, Answer, Document
start = time.time()
eval_labels = [
   MultiLabel(
       labels=[
           Label(
               query=data_test["data"][i]["paragraphs"][0]['qas'][0]['question'],
               answer=Answer(
                   answer=data_test["data"][i]["paragraphs"][0]['qas'][0]['answers'][0]['text'],
               ),
               document=Document(
                   id=data_test["data"][i]["paragraphs"][0]['document_id'],
                   content_type="text",
                   content=data_test["data"][i]["paragraphs"][0]['context']
               ),
               is_correct_answer=True,
               is_correct_document=True,
               origin="gold-label"
           )
       ]
   ) for i in range(len(data_test["data"]))
]

docs = [[Document(
                   id=data_test["data"][i]["paragraphs"][0]['document_id'],
                   content_type="text",
                   content=data_test["data"][i]["paragraphs"][0]['context']
               ) for i in range(len(data_test["data"])) ]]


# Similar to pipeline.run() we can execute pipeline.eval()
eval_result = pipe.eval(labels=eval_labels, params={"Retriever": {"top_k": 3}})
print(f"60 evaluations finished in : {int(time.time()-start)}s")

Inferencing Samples:   0%|          | 0/1 [00:00<?, ? Batches/s]/usr/local/lib/python3.7/dist-packages/haystack/modeling/model/prediction_head.py:485: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  start_indices = flat_sorted_indices // max_seq_len
Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00, 30.72 Batches/s]


60 evaluations finished in : 26s


In [ ]:
26/60

0.43333333333333335

In [ ]:
reader_result = eval_result["Reader"]
reader_result.head()


,multilabel_id,query,filters,gold_answers,answer,context,exact_match,f1,rank,document_id,gold_document_ids,offsets_in_document,gold_offsets_in_documents,type,node,eval_mode
0,-677990051653884372,Que sous-entendent les scénarios 100% renouvelables ou de prolongation des c...,b'null',[grand nombre de prérequis techniques critiques soient respectés à court terme],un grand nombre de prérequis techniques critiques soient respectés à court t...,ls au-delà de 60 ans impliquent qu’un grand nombre de prérequis techniques c...,0.0,0.956522,1.0,312295387148fc8fa6015cdae4e1c14,[15],"[{'start': 244, 'end': 324}]",[],answer,Reader,integrated
1,-677990051653884372,Que sous-entendent les scénarios 100% renouvelables ou de prolongation des c...,b'null',[grand nombre de prérequis techniques critiques soient respectés à court terme],«volume exceptionnel de travaux».,"qué que la prolongation au-delà de 40 ans, pour laquelle elle a rendu un avi...",0.0,0.133333,2.0,8dd9975d24d0b368950611dbe67cf30a,[15],"[{'start': 378, 'end': 411}]",[],answer,Reader,integrated
2,-677990051653884372,Que sous-entendent les scénarios 100% renouvelables ou de prolongation des c...,b'null',[grand nombre de prérequis techniques critiques soient respectés à court terme],"renouvelables, ou celui nécessitant la prolongation","Les scénarios à très hautes parts d’énergies renouvelables, ou celui nécessi...",0.0,0.000000,3.0,b4431cd74a2b7f1e2ffe3d3c9638f213,[15],"[{'start': 45, 'end': 96}]",[],answer,Reader,integrated
0,-5483248663165921264,De quoi s'affranchissent les scénarios type N2 ?,b'null',[plusieurs paris techniques et industriels],techniques et industriels,arios de type N2 permettent de s’affranchir de plusieurs paris techniques et...,0.0,0.750000,1.0,e9165f570093198da913fea53deb2254,[24],"[{'start': 71, 'end': 96}]",[],answer,Reader,integrated
1,-5483248663165921264,De quoi s'affranchissent les scénarios type N2 ?,b'null',[plusieurs paris techniques et industriels],volontarisme pour l’installation des énergies renouvelables,s pleinement concrétisées dans les faits : les scénarios N2 et N03 supposent...,0.0,0.000000,2.0,d565b75cbe9ebdf19f3c3eb4799c4863,[24],"[{'start': 508, 'end': 567}]",[],answer,Reader,integrated


In [ ]:
# reader_result.to_csv("/content/drive/MyDrive/Projet Option - ChatBot/3 - Scripts/eval_CamembertV2.csv")
#reader_result.to_csv("/content/drive/MyDrive/Projet Option - ChatBot/3 - Scripts/eval_Camembert.csv")
reader_result.to_csv("/content/drive/MyDrive/Projet Option - ChatBot/3 - Scripts/eval_xlm.csv")

In [13]:
reader_result["f1"].iloc[3*2+1]

NameError: ignored

In [14]:
retriever_result = eval_result["Retriever"]
retriever_result.head()

,multilabel_id,query,filters,gold_document_contents,content,gold_id_match,answer_match,gold_id_or_answer_match,rank,document_id,gold_document_ids,type,node,eval_mode
0,-4574023199644543112,Que sous-entendent les scénarios 100% renouvelables ou de prolongation des c...,b'null',[Si les défis technologiques et de R&D associés apparaissent «dépassables» d...,"Les scénarios à très hautes parts d’énergies renouvelables, ou celui nécessi...",0.0,0.0,0.0,1.0,b4431cd74a2b7f1e2ffe3d3c9638f213,[15],document,Retriever,integrated
1,-4574023199644543112,Que sous-entendent les scénarios 100% renouvelables ou de prolongation des c...,b'null',[Si les défis technologiques et de R&D associés apparaissent «dépassables» d...,Si les défis technologiques et de R&D associés apparaissent «dépassables» da...,0.0,1.0,1.0,2.0,312295387148fc8fa6015cdae4e1c14,[15],document,Retriever,integrated
2,-4574023199644543112,Que sous-entendent les scénarios 100% renouvelables ou de prolongation des c...,b'null',[Si les défis technologiques et de R&D associés apparaissent «dépassables» d...,Cinq des six scénarios des «Futurs énergétiques 2050» reposent sur une explo...,0.0,0.0,0.0,3.0,8dd9975d24d0b368950611dbe67cf30a,[15],document,Retriever,integrated
0,-7532756540141021476,De quoi s'affranchissent les scénarios type N2 ?,b'null',[Les scénarios de type N2 permettent de s’affranchir de plusieurs paris tech...,Les scénarios de type N2 permettent de s’affranchir de plusieurs paris techn...,0.0,1.0,1.0,1.0,e9165f570093198da913fea53deb2254,[24],document,Retriever,integrated
1,-7532756540141021476,De quoi s'affranchissent les scénarios type N2 ?,b'null',[Les scénarios de type N2 permettent de s’affranchir de plusieurs paris tech...,"Les scénarios N2 et N03, qui reposent sur le maintien durable d’un parc nucl...",0.0,0.0,0.0,2.0,d565b75cbe9ebdf19f3c3eb4799c4863,[24],document,Retriever,integrated


In [20]:
retriever_result[retriever_result["rank"]==1.0]["gold_id_or_answer_match"].sum()/len(retriever_result[retriever_result["rank"]==1.0])

0.36666666666666664

In [15]:
retriever_result.iloc[0]

multilabel_id                                                                         -4574023199644543112
query                      Que sous-entendent les scénarios 100% renouvelables ou de prolongation des c...
filters                                                                                            b'null'
gold_document_contents     [Si les défis technologiques et de R&D associés apparaissent «dépassables» d...
content                    Les scénarios à très hautes parts d’énergies renouvelables, ou celui nécessi...
gold_id_match                                                                                          0.0
answer_match                                                                                           0.0
gold_id_or_answer_match                                                                                0.0
rank                                                                                                   1.0
document_id                          

In [ ]:
import numpy as np
all_f1 = []
for i in range(len(reader_result)//3):
  f1 = max(reader_result["f1"].iloc[3*i], reader_result["f1"].iloc[3*i+1], reader_result["f1"].iloc[3*i+2])
  all_f1.append(f1)
np.mean(all_f1)

0.37849922625677107

In [ ]:
import numpy as np
f1_first = []
f1_second = []
f1_third = []

for i in range(len(reader_result)//3):
    f1_first.append(reader_result["f1"].iloc[3*i])
    f1_second.append(reader_result["f1"].iloc[3*i+1])
    f1_third.append(reader_result["f1"].iloc[3*i+2])
print(f"La moyenne du f1 score de la première réponse est {np.mean(f1_first)}")
print(f"La moyenne du f1 score de la deuxième réponse est {np.mean(f1_second)}")
print(f"La moyenne du f1 score de la deuxième réponse est {np.mean(f1_third)}")

La moyenne du f1 score de la première réponse est 0.24524329826585775
La moyenne du f1 score de la deuxième réponse est 0.15367713689846546
La moyenne du f1 score de la deuxième réponse est 0.04712201303167415


**CamemBERT etalab** 

f1 moyen pour le meilleur dans les tops 3 réponses = **0.4410** \
La moyenne du f1 score de la première réponse est 0.30833837907250766 \
La moyenne du f1 score de la deuxième réponse est 0.15077226166061355 \
La moyenne du f1 score de la deuxième réponse est 0.08216946976350933 \


**CamemBERT etalab fine-tuné** 

f1 moyen pour le meilleur dans les tops 3 réponses = **0.4555** \
La moyenne du f1 score de la première réponse est 0.34376325079855674\
La moyenne du f1 score de la deuxième réponse est 0.14293577103049535\
La moyenne du f1 score de la deuxième réponse est 0.09896793972304943 \

**XLM-RoBERTa fine tune sur Texas-squad-fr** 

f1 moyen pour le meilleur dans les tops 3 réponses = **0.3785** \
La moyenne du f1 score de la première réponse est 0.24524329826585775 \
La moyenne du f1 score de la deuxième réponse est 0.15367713689846546 \
La moyenne du f1 score de la deuxième réponse est 0.04712201303167415




In [ ]:
for data in data_test:
  prediction = pipe.run(
      query="", 
      params={"Reader": {"top_k": 3}}
  )

# print the result 
print_answers(prediction, details="medium")

In [ ]:
for j in range(len(data_test["data"])):
    count=0
    for i in range(len(dict_rapport)):
    if dict_rapport[i]["content"]==data_test["data"][j]["paragraphs"][0]["context"]:
      #print(dict_rapport[i]["content"])
      count+=1
    if count==0:
    # print(dict_rapport[i]["content"])
    print("\n", data_test["data"][j]["paragraphs"][0]["context"])
#dict_rapport[15]

In [29]:
data_test["data"][0]["paragraphs"][0]# ['document_id']

{'context': 'Si les défis technologiques et de R&D associés apparaissent «dépassables» dans les décennies à venir, les scénarios «100% renouvelable» ou fondés sur la prolongation à long terme des réacteurs nucléaires actuels au-delà de 60 ans impliquent qu’un grand nombre de prérequis techniques critiques soient respectés à court terme. Or rien ne le garantit en l’état. Décider de ces scénarios aujourd’hui, ou renoncer au principe de diversification technologique dans le mix de production électrique, soulève donc un risque de non-atteinte de l’objectif de neutralité carbone à la date rapprochée de 2050.',
 'document_id': 15,
 'qas': [{'answers': [{'answer_category': 'SHORT',
     'answer_id': 5015,
     'answer_start': 247,
     'document_id': 15,
     'question_id': 1015,
     'text': 'grand nombre de prérequis techniques critiques soient respectés à court terme'}],
   'id': 1015,
   'is_impossible': False,
   'question': 'Que sous-entendent les scénarios 100% renouvelables ou de prol

In [ ]:
document_store = ElasticsearchDocumentStore(host="localhost", username="", password="", index="document")
report_path = "/content/drive/MyDrive/Projet Option - ChatBot/0 - Biblio/rapport_rte_v2.json"

# Import text 
with open(report_path) as json_file:
    dict_rapport = json.load(json_file)["data"]

# write document on ElasticSearch server
document_store.write_documents(dict_rapport)

## Initalize Retriever, Reader,  & Pipeline
retriever = ElasticsearchRetriever(document_store=document_store)
reader = FARMReader(model_name_or_path="/content/drive/MyDrive/Projet Option - ChatBot/3 - Scripts/camembertV2", use_gpu=True)
# reader = FARMReader(model_name_or_path="saattrupdan/xlmr-base-texas-squad-fr", use_gpu=True)

pipe = ExtractiveQAPipeline(reader, retriever)